# 데이터 전처리

라이브러리 import

In [1]:
import pandas as pd
# import dataframe_image as dfi
import numpy as np
import time, datetime, random
import os, itertools
import matplotlib.pyplot as plt
import seaborn as sns

import os 
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) # FutureWarning 제거

import matplotlib.pyplot as plt #  matplotlib 시각화 패키지 
import seaborn as sns
%matplotlib inline

# os에 따른 Font 깨짐 제거를 위해 Font 지정 
import os 
if os.name == 'nt' :  # Windows OS 
    font_family = "Malgun Gothic"
else: # MAC OS 
    font_family = "AppleGothic"
    
# - 값이 깨지는 문제 해결을 위해 파라미터 값 설정 
sns.set(font=font_family, rc ={"axes.unicode_minus" : False})

In [2]:
저장 = '../전처리데이터/구매내역/'

- 김해사랑상품권 원본 데이터(2019.08 ~ 2022.08) 결합(pd.merge)후 전처리 진행
- 김해사랑상품권 데이터: '구매', 김해 공공배달앱상품권 데이터: '배달앱

In [ ]:
# 이용자번호, 거래번호는 고유한 값인데 csv로 저장 시 뒷자리 숫자가 생략된 채 저장이 되는 경우가 있어 str로 데이터 타입 변경
구매_ = 구매_.astype({'이용자번호':'str'})
구매_ = 구매_.astype({'거래번호':'str'})
# 원본 데이터의 [구매일]에서 [구매년도, 구매월, 구매일(d)를 따로 추출한 후 새로운 컬럼 생성 → .str.split(구분기호, expand=True)[구분 된 문자 인덱스 번호] 사용]
# [구매시]에서 [구매시(h)] 추출한 것도 같은 원리
구매['구매년도']=구매['구매일'].str.split('-', expand=True)[0]
구매['구매월']=구매['구매일'].str.split('-', expand=True)[1]
구매['구매일(d)']=구매['구매일'].str.split('-', expand=True)[2]
구매['구매시(h)']=구매['구매시'].str.split(':', expand=True)[0]

In [ ]:
# 생년월일에서 태어난 연도만 추출 한 후 올해를 기준으로 한국식 나이 계산
구매 = 구매.astype({'생년월일':'str'}) # 데이터에서는 생년월일이 실수로 되어 있어서 생년월일을 문자타입으로 변경
구매['나이1']=구매['생년월일'].str.split('.', expand=True)[0] # . 앞에만 추출
구매['나이2'] = 구매['나이1'].str[0:4] # 태어난 연도만 추출
구매 = 구매.astype({'나이2':'float'}) # 나이 계산을 위해 다시 실수로 데이터 타입 변경
구매['연령'] = (2022-구매['나이2'])+1 # 한국식 나이는 +1을 해줘야 하니까
구매 = 구매.astype({'연령':'str'}) # 연령대 컬럼 추가를 위해 다시 문자 타입으로 변경
구매['연령대'] = 구매['연령'].str[0]+'0대' # 연령에서 십의자리 숫자만 추출 후 연령대 컬럼 생성
구매 = 구매.drop(['나이1', '나이2'], axis = 1) # 필요없는 컬럼 삭제
구매

In [3]:
구매 = pd.read_csv('구매_김해사랑상품권_진짜.csv', encoding = 'cp949')

In [4]:
구매

,Unnamed: 0,거래번호,상품권,구매처,구매일,구매시,이용자번호,이용자명,전화번호,총금액(A+B),고객구매금(A),지원금(B),성별,생년월일,구매년도,구매월,구매일(d),구매시(h),나이,나이대
0,0,PKP0000515841,김해사랑상품권,부산은행,2019-12-31,23:42,19123106325178,호선애,1.047955e+09,500000,450000,50000,F,199201.0,2019,12,31,23,31.0,30대
1,1,PKP0000515704,김해사랑상품권,농협은행,2019-12-31,23:15,19121705484159,최은하,1.027332e+09,100000,90000,10000,F,198103.0,2019,12,31,23,42.0,40대
2,2,PKP0000515648,김해사랑상품권,농협은행,2019-12-31,23:02,19100202556809,박민혁,1.066161e+09,450000,405000,45000,M,198712.0,2019,12,31,23,36.0,30대
3,3,PKP0000515642,김해사랑상품권,쿠콘,2019-12-31,23:02,19123106323197,서봉근,1.091017e+09,300000,270000,30000,M,198004.0,2019,12,31,23,43.0,40대
4,4,PKP0000515290,김해사랑상품권,경남은행,2019-12-31,21:59,19123106319012,안규나,1.075267e+09,100000,90000,10000,F,195908.0,2019,12,31,21,64.0,60대
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1040694,1040694,PKP0204040632,김해사랑상품권,비즈플레이,2022-07-01,11:01,20090936355925,구미정,1.092592e+09,200000,180000,20000,F,197809.0,2022,7,1,11,45.0,40대
1040695,1040695,PKP0204040625,김해사랑상품권,비즈플레이,2022-07-01,11:01,20041957483163,김선희,1.085011e+09,200000,180000,20000,F,198206.0,2022,7,1,11,41.0,40대
1040696,1040696,PKP0204040607,김해사랑상품권,농협은행,2022-07-01,11:01,20042365732963,박인숙,1.024182e+09,200000,180000,20000,F,197902.0,2022,7,1,11,44.0,40대
1040697,1040697,PKP0204040605,김해사랑상품권,농협은행,2022-07-01,11:01,19082700301763,임은주,1.093295e+09,200000,180000,20000,F,197711.0,2022,7,1,11,46.0,40대


# 각 월 최대한도금액 인원 구하기


- 김해시는 각 월마다 최대구마한도 존재, 최대구매한도를 채워서 구매한 인원 추출
- df.drop_duplocates('중복을 제거할 열 이름') 함수를 사용하여 이용자번호 중복을 제거하고 인원 파악

In [ ]:
#2021년
구매_2021_1월_50만원 = 구매[(구매['구매년도']==2021) & (구매['구매월']==1) & (구매['총금액(A+B)']==500000)]
a = len(구매_2021_1월_50만원.drop_duplicates('이용자번호'))

구매_2021_2월_70만원 = 구매[(구매['구매년도']==2021) & (구매['구매월']==2) & (구매['총금액(A+B)']==700000)]
b = len(구매_2021_2월_70만원.drop_duplicates('이용자번호'))

구매_2021_3월_30만원 = 구매[(구매['구매년도']==2021) & (구매['구매월']==3) & (구매['총금액(A+B)']==300000)]
c = len(구매_2021_3월_30만원.drop_duplicates('이용자번호'))

구매_2021_4월_30만원 = 구매[(구매['구매년도']==2021) & (구매['구매월']==4) & (구매['총금액(A+B)']==300000)]
d = len(구매_2021_4월_30만원.drop_duplicates('이용자번호'))

구매_2021_5월_30만원 = 구매[(구매['구매년도']==2021) & (구매['구매월']==5) & (구매['총금액(A+B)']==300000)]
e = len(구매_2021_5월_30만원.drop_duplicates('이용자번호'))

구매_2021_6월_30만원 = 구매[(구매['구매년도']==2021) & (구매['구매월']==6) & (구매['총금액(A+B)']==300000)]
f = len(구매_2021_6월_30만원.drop_duplicates('이용자번호'))

구매_2021_7월_20만원 = 구매[(구매['구매년도']==2021) & (구매['구매월']==7) & (구매['총금액(A+B)']==200000)]
g = len(구매_2021_7월_20만원.drop_duplicates('이용자번호'))

구매_2021_8월_20만원 = 구매[(구매['구매년도']==2021) & (구매['구매월']==8) & (구매['총금액(A+B)']==200000)]
h = len(구매_2021_8월_20만원.drop_duplicates('이용자번호'))

구매_2021_9월_30만원 = 구매[(구매['구매년도']==2021) & (구매['구매월']==9) & (구매['총금액(A+B)']==300000)]
i = len(구매_2021_9월_30만원.drop_duplicates('이용자번호'))

구매_2021_10월_20만원 = 구매[(구매['구매년도']==2021) & (구매['구매월']==10) & (구매['총금액(A+B)']==200000)]
j = len(구매_2021_10월_20만원.drop_duplicates('이용자번호'))

구매_2021_11월_30만원 = 구매[(구매['구매년도']==2021) & (구매['구매월']==11) & (구매['총금액(A+B)']==300000)]
k = len(구매_2021_11월_30만원.drop_duplicates('이용자번호'))

구매_2021_12월_20만원 = 구매[(구매['구매년도']==2021) & (구매['구매월']==12) & (구매['총금액(A+B)']==200000)]
l = len(구매_2021_12월_20만원.drop_duplicates('이용자번호'))

#2022년
구매_2022_1월_20만원 = 구매[(구매['구매년도']==2022) & (구매['구매월']==1) & (구매['총금액(A+B)']==200000)]
m = len(구매_2022_1월_20만원.drop_duplicates('이용자번호'))

구매_2022_2월_20만원 = 구매[(구매['구매년도']==2022) & (구매['구매월']==2) & (구매['총금액(A+B)']==200000)]
n = len(구매_2022_2월_20만원.drop_duplicates('이용자번호'))

구매_2022_3월_20만원 = 구매[(구매['구매년도']==2022) & (구매['구매월']==3) & (구매['총금액(A+B)']==200000)]
o = len(구매_2022_3월_20만원.drop_duplicates('이용자번호'))

구매_2022_4월_20만원 = 구매[(구매['구매년도']==2022) & (구매['구매월']==4) & (구매['총금액(A+B)']==200000)]
p = len(구매_2022_4월_20만원.drop_duplicates('이용자번호'))

구매_2022_5월_20만원 = 구매[(구매['구매년도']==2022) & (구매['구매월']==5) & (구매['총금액(A+B)']==200000)]
q = len(구매_2022_5월_20만원.drop_duplicates('이용자번호'))

구매_2022_6월_20만원 = 구매[(구매['구매년도']==2022) & (구매['구매월']==6) & (구매['총금액(A+B)']==200000)]
r = len(구매_2022_6월_20만원.drop_duplicates('이용자번호'))

구매_2022_7월_20만원 = 구매[(구매['구매년도']==2022) & (구매['구매월']==7) & (구매['총금액(A+B)']==200000)]
s = len(구매_2022_7월_20만원.drop_duplicates('이용자번호'))

구매_2022_8월_20만원 = 구매[(구매['구매년도']==2022) & (구매['구매월']==8) & (구매['총금액(A+B)']==200000)]
t = len(구매_2022_8월_20만원.drop_duplicates('이용자번호'))


print('2021년 1월 최대한도 구매 인원:', a)
print('2021년 2월 최대한도 구매 인원:', b)
print('2021년 3월 최대한도 구매 인원:', c)
print('2021년 4월 최대한도 구매 인원:', d)
print('2021년 5월 최대한도 구매 인원:', e)
print('2021년 6월 최대한도 구매 인원:', f)
print('2021년 7월 최대한도 구매 인원:', g)
print('2021년 8월 최대한도 구매 인원:', h)
print('2021년 9월 최대한도 구매 인원:', i)
print('2021년 10월 최대한도 구매 인원:',j)
print('2021년 11월 최대한도 구매 인원:', k)
print('2021년 12월 최대한도 구매 인원:', l)
print('2022년 1월 최대한도 구매 인원:', m)
print('2022년 2월 최대한도 구매 인원:', n)
print('2022년 3월 최대한도 구매 인원:', o)
print('2022년 4월 최대한도 구매 인원:', p)
print('2022년 5월 최대한도 구매 인원:', q)
print('2022년 6월 최대한도 구매 인원:', r)
print('2022년 7월 최대한도 구매 인원:', s)
print('2022년 8월 최대한도 구매 인원:', t)

2021년 1월 최대한도 구매 인원: 15687
2021년 2월 최대한도 구매 인원: 8352
2021년 3월 최대한도 구매 인원: 17859
2021년 4월 최대한도 구매 인원: 18189
2021년 5월 최대한도 구매 인원: 18454
2021년 6월 최대한도 구매 인원: 18753
2021년 7월 최대한도 구매 인원: 46815
2021년 8월 최대한도 구매 인원: 46874
2021년 9월 최대한도 구매 인원: 33607
2021년 10월 최대한도 구매 인원: 42043
2021년 11월 최대한도 구매 인원: 34438
2021년 12월 최대한도 구매 인원: 34021
2022년 1월 최대한도 구매 인원: 38472
2022년 2월 최대한도 구매 인원: 29251
2022년 3월 최대한도 구매 인원: 38647
2022년 4월 최대한도 구매 인원: 38863
2022년 5월 최대한도 구매 인원: 43605
2022년 6월 최대한도 구매 인원: 38985
2022년 7월 최대한도 구매 인원: 39010
2022년 8월 최대한도 구매 인원: 48556


# 구매내역 관련 각 통계들

- 통계를 낼 때 df.groupby('열이름')을 자주 사용

In [19]:
# 연도별 구매내역
연도별_구매내역 = 구매.groupby('구매년도').agg({'거래번호':'count','총금액(A+B)':'sum', '고객구매금(A)':'sum', '지원금(B)':'sum'})
연도별_구매내역.rename(columns={'거래번호':'구매건수'}, inplace=True)
연도별_구매내역

,구매건수,총금액(A+B),고객구매금(A),지원금(B)
구매년도,,,,
2019,4045,712965000,641733500,71231500
2020,270992,61070505000,54992054500,6078450500
2021,429039,102958502000,92662651800,10295850200
2022,336623,65147356000,58632620400,6514735600


In [20]:
# 월별 구매내역
월별_구매내역 = 구매.groupby(['구매년도', '구매월']).agg({'거래번호':'count', '총금액(A+B)':'sum', '고객구매금(A)':'sum', '지원금(B)':'sum'})
월별_구매내역.rename(columns={'거래번호':'구매건수'}, inplace=True)
월별_구매내역

구매건수     총금액(A+B)     고객구매금(A)      지원금(B)
구매년도 구매월                                             
2019 8      348     59980000     54009900     5970100
     9      789    134260000    120834000    13426000
     10     851    165095000    148612600    16482400
     11     912    143080000    128782000    14298000
     12    1145    210550000    189495000    21055000
2020 1     3093    710075000    639067500    71007500
     2     4549   1123240000   1011166000   112074000
     3     5445   1255760000   1130920000   124840000
     4    11059   2587200000   2330194500   257005500
     5    14604   4147255000   3735461500   411793500
     6    17749   4839140000   4358124000   481016000
     7    27647   6396375000   5760732500   635642500
     8    38571   8567445000   7717937000   849508000
     9    50089  10550740000   9504213000  1046527000
     10   53445   9903170000   8913144000   990026000
     11   44736  10989790000   9890811000  1098979000
     12       5       315000       283500       31500
2021 1    30076  10197135000   9177421500  1019713500
     2    17260   8088100000   7279290000   808810000
     3    23558   6053410000   5448069000   605341000
     4    22929   6043910000   5439519000   604391000
     5    22261   6027285000   5424556500   602728500
     6    21405   6024695000   5422225500   602469500
     7    55041  10084160000   9075744000  1008416000
     8    54968  10076298000   9068668200  1007629800
     9    47411  12076499000  10868849100  1207649900
     10   49617   9061640000   8155476000   906164000
     11   46814  12075630000  10868067000  1207563000
     12   37699   7149740000   6434766000   714974000
2022 1    41930   8026825000   7224142500   802682500
     2    30886   6013191000   5411871900   601319100
     3    41652   8018030000   7216227000   801803000
     4    41340   8014310000   7212879000   801431000
     5    46628   9016705000   8115034500   901670500
     6    41186   8014080000   7212672000   801408000
     7    41137   8013855000   7212469500   801385500
     8    51864  10030360000   9027324000  1003036000

In [38]:
#성별 통계
구매_성별 = 구매.groupby('성별').agg({'상품권':'count', '총금액(A+B)':'sum'})
구매_성별.rename(columns={'상품권':'구매건수'}, inplace=True)
구매_성별.reset_index(inplace=True)
구매_성별['전체 건수 대비 비율'] = (구매_성별['구매건수']/구매_성별['구매건수'].sum())*100
구매_성별['전체 금액 대비 비율'] = (구매_성별['총금액(A+B)']/구매_성별['총금액(A+B)'].sum())*100
구매_성별

,성별,구매건수,총금액(A+B),전체 건수 대비 비율,전체 금액 대비 비율
0,F,669584,144542955000,64.367295,62.898261
1,M,370671,85261418000,35.632705,37.101739


In [40]:
# 연령별 통계
구매_나이대 = 구매.groupby('나이대').agg({'상품권':'count', '총금액(A+B)':'sum'})
구매_나이대.drop(index='-0대', inplace=True)
구매_나이대.drop(index='n0대', inplace=True)
구매_나이대.rename(columns={'상품권':'구매건수'}, inplace=True)
구매_나이대.reset_index(inplace=True)
구매_나이대['전체 건수 대비 비율'] = (구매_나이대['구매건수']/구매_나이대['구매건수'].sum())*100
구매_나이대['전체 금액 대비 비율'] = (구매_나이대['총금액(A+B)']/구매_나이대['총금액(A+B)'].sum())*100
구매_나이대

,나이대,구매건수,총금액(A+B),전체 건수 대비 비율,전체 금액 대비 비율
0,10대,11516,2565325000,1.107054,1.116328
1,20대,51148,10339420000,4.916952,4.499305
2,30대,196181,41541051000,18.859242,18.077016
3,40대,488275,111242272000,46.938777,48.408221
4,50대,221227,48313092000,21.266960,21.023940
5,60대,59063,12844088000,5.677835,5.589237
6,70대,11396,2619405000,1.095519,1.139861
7,80대,1355,317080000,0.130259,0.137981
8,90대,77,18640000,0.007402,0.008111


In [ ]:
# 구매처 통계
구매처_통계 = 구매.groupby('구매처', as_index = False).agg({'거래번호':'count', '총금액(A+B)':'sum'})
구매처_통계.rename(columns={'거래번호':'구매건수'}, inplace=True)

In [ ]:
구매처_통계['건수 대비 비율'] = round((구매처_통계['구매건수']/구매처_통계['구매건수'].sum())*100,2)
구매처_통계['금액 대비 비율'] = round((구매처_통계['총금액(A+B)']/구매처_통계['총금액(A+B)'].sum())*100,2)
구매처_통계.sort_values(by=['총금액(A+B)'], ascending=[False], inplace = True)
구매처_통계.reset_index(inplace=True)

In [ ]:
구매처_통계.drop(['level_0', 'index'], axis=1, inplace = True)
구매처_통계

,구매처,구매건수,총금액(A+B),건수 대비 비율,금액 대비 비율
0,비즈플레이,412671,90022921000,39.65,39.16
1,비즈플레이(경남),351222,78499483000,33.75,34.15
2,농협은행,142785,31227741000,13.72,13.58
3,쿠콘,70849,16867405000,6.81,7.34
4,경남은행,24727,5639137000,2.38,2.45
5,부산은행,7903,1840781000,0.76,0.80
6,PAYCO,7798,1373710000,0.75,0.60
7,NH농협은행(카드형),6897,1265349000,0.66,0.55
8,갤럭시아컴즈,4783,982130000,0.46,0.43
9,신한은행,2258,469781000,0.22,0.20


# 월 최대구매한도 20만원 이었던 달 분석

-  2021년 7, 8, 10, 12월
- 2022년 1 ~ 8월

In [5]:
구매_ = 구매

In [6]:
구매_ = 구매_.astype({'구매년도':'str'})
구매_ = 구매_.astype({'구매월':'str'})
구매_ = 구매_.astype({'구매일(d)':'str'})

In [7]:
# 월 최대구매한도 20만원 달만 추출하ㅣ하기 위해서
구매_['구매년월'] = 구매_['구매년도'] + '-' + 구매_['구매월']

In [8]:
구매_한도_20만원 = 구매_[(구매_['구매년월']=='2021-7') | (구매_['구매년월']=='2021-8')| (구매_['구매년월']=='2021-10')| (구매_['구매년월']=='2021-12') |
                        (구매_['구매년월']=='2022-1') | (구매_['구매년월']=='2022-2') | (구매_['구매년월']=='2022-3') | (구매_['구매년월']=='2022-4') | 
                        (구매_['구매년월']=='2022-5') | (구매_['구매년월']=='2022-6') | (구매_['구매년월']=='2022-7') | (구매_['구매년월']=='2022-8')]

In [9]:
구매_한도_20만원

,Unnamed: 0,거래번호,상품권,구매처,구매일,구매시,이용자번호,이용자명,전화번호,총금액(A+B),...,지원금(B),성별,생년월일,구매년도,구매월,구매일(d),구매시(h),나이,나이대,구매년월
415515,415515,PKP0103610278,김해사랑상품권,비즈플레이(경남),2021-08-15,18:37,20120163760034,김은하,1.049464e+09,200000,...,20000,F,197602.0,2021,8,15,18,47.0,40대,2021-8
415516,415516,PKP0103567297,김해사랑상품권,농협은행,2021-08-15,14:08,19082900660360,김명희,1.030892e+09,30000,...,3000,F,196511.0,2021,8,15,14,58.0,50대,2021-8
415517,415517,PKP0103196858,김해사랑상품권,비즈플레이,2021-08-13,18:15,21062406984361,류성훈,1.038472e+09,25000,...,2500,M,197211.0,2021,8,13,18,51.0,50대,2021-8
415518,415518,PKP0103196570,김해사랑상품권,비즈플레이,2021-08-13,18:14,21062406984361,류성훈,1.038472e+09,15000,...,1500,M,197211.0,2021,8,13,18,51.0,50대,2021-8
415519,415519,PKP0103160382,김해사랑상품권,비즈플레이,2021-08-13,15:10,21010776106714,이혜정,1.085258e+09,200000,...,20000,F,195703.0,2021,8,13,15,66.0,60대,2021-8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1040694,1040694,PKP0204040632,김해사랑상품권,비즈플레이,2022-07-01,11:01,20090936355925,구미정,1.092592e+09,200000,...,20000,F,197809.0,2022,7,1,11,45.0,40대,2022-7
1040695,1040695,PKP0204040625,김해사랑상품권,비즈플레이,2022-07-01,11:01,20041957483163,김선희,1.085011e+09,200000,...,20000,F,198206.0,2022,7,1,11,41.0,40대,2022-7
1040696,1040696,PKP0204040607,김해사랑상품권,농협은행,2022-07-01,11:01,20042365732963,박인숙,1.024182e+09,200000,...,20000,F,197902.0,2022,7,1,11,44.0,40대,2022-7
1040697,1040697,PKP0204040605,김해사랑상품권,농협은행,2022-07-01,11:01,19082700301763,임은주,1.093295e+09,200000,...,20000,F,197711.0,2022,7,1,11,46.0,40대,2022-7


In [ ]:
# 구매금액
구매_한도_20만원['총금액(A+B)'].sum()

101519194000

In [ ]:
# 구매인원
len(구매_한도_20만원.drop_duplicates('이용자번호'))

101558

In [ ]:
# 구매한도 20만원 시기 구매자 정보 추출
구매_한도_20만원_구매자정보=구매_한도_20만원.groupby(['이용자번호', '성별', '나이']).agg({'구매처':'count', '총금액(A+B)':'sum'})
구매_한도_20만원_구매자정보.sort_values(by=['총금액(A+B)'], ascending=[False], inplace = True)
구매_한도_20만원_구매자정보.reset_index(inplace=True)
구매_한도_20만원_구매자정보

In [ ]:
구매_한도_20만원_구매자정보.rename(columns={'구매처':'구매건수'}, inplace=True)

In [ ]:
구매_한도_20만원_구매자정보

,이용자번호,성별,나이,구매건수,총금액(A+B)
0,20092140309332,F,46.0,24,2730000
1,20013110463479,F,48.0,15,2620000
2,20060408287947,F,46.0,13,2600000
3,20042771039589,F,44.0,13,2600000
4,20071017459834,F,43.0,13,2600000
...,...,...,...,...,...
101510,19100802706220,M,40.0,1,5000
101511,20052202168396,F,28.0,1,5000
101512,21100561828664,F,36.0,1,5000
101513,19081700011038,F,41.0,1,5000


## 연령대별 구매금액 상위 이용자 추출

In [ ]:
구매_한도_20만원_연령별 = 구매_한도_20만원.groupby(['나이대', '이용자번호', '나이', '성별']).agg({'구매처':'count', '총금액(A+B)':'sum'})
구매_한도_20만원_연령별.drop('-0대', axis = 0, inplace = True)

In [ ]:
구매_한도_20만원_연령별.rename(columns={'구매처':'구매건수'}, inplace=True)

In [ ]:
구매_한도_20만원_연령별

구매건수  총금액(A+B)
나이대 이용자번호          나이   성별                
10대 19082700344335 19.0 F      3    600000
    19110203420153 18.0 F      4    800000
    19120905036953 19.0 F      9   1650000
    19122906152502 18.0 M      7   1400000
    20010906923163 19.0 M      1    200000
...                          ...       ...
90대 21070110069510 97.0 F      5   1000000
    21070210858435 90.0 F      3    600000
    21070612371736 90.0 F      1    200000
    21071014106813 93.0 F      2    310000
    22060261009650 90.0 M      2    400000

[101513 rows x 2 columns]

In [ ]:
# 멀티 인덱스 이기 때문에 '총금액(A+B)'을 기준으로 새로 인덱스를 주고 'group_keys = False'로 주어서 인데스를 기본값으로 설정
group = 구매_한도_20만원_연령별['총금액(A+B)'].groupby(level=0, group_keys=False)

In [ ]:
pd.DataFrame(group)

,0,1
0,10대,나이대 이용자번호 나이 성별 10대 19082700344...
1,20대,나이대 이용자번호 나이 성별 20대 19082000052...
2,30대,나이대 이용자번호 나이 성별 30대 19081800011...
3,40대,나이대 이용자번호 나이 성별 40대 19081700008...
4,50대,나이대 이용자번호 나이 성별 50대 19081900016...
5,60대,나이대 이용자번호 나이 성별 60대 19081900032...
6,70대,나이대 이용자번호 나이 성별 70대 19082600270...
7,80대,나이대 이용자번호 나이 성별 80대 20011607614...
8,90대,나이대 이용자번호 나이 성별 90대 19102203088...


In [ ]:
# 그 후 '시리즈.nlargest(n, keep='all') 실시해서 상위 이용자 추출'
# keep = 'all' → 동등한 값을 모두 표시
pd.DataFrame(group.nlargest(3, keep='all'))

총금액(A+B)
나이대 이용자번호          나이   성별          
10대 20052805543559 19.0 M    2400000
    20072524302478 18.0 M    2400000
    20082131133369 19.0 M    2400000
    20092945722688 19.0 M    2400000
    20101550321524 19.0 M    2400000
...                              ...
80대 21010976659470 82.0 M    2400000
    21063009428583 84.0 F    2400000
90대 20082431816859 90.0 F    2200000
    21050351136205 92.0 F    1200000
    21070110069510 97.0 F    1000000

[255 rows x 1 columns]

In [ ]:
# 연령별 총금액(A+B) 상위 1명 이용자 추출 - 방법2

In [10]:
 구매_한도_20만원_연령별_2 = 구매_한도_20만원.groupby(['나이대', '이용자번호', '나이', '성별']).agg({'구매처':'count', '총금액(A+B)':'sum'}).sort_values(by=['나이대','총금액(A+B)'], ascending=False).groupby('나이대').head(5)

In [12]:
구매_한도_20만원_연령별_2.rename(columns={'거래건수':'구매건수'}, inplace=True)
구매_한도_20만원_연령별_2.drop('-0대', axis = 0, inplace = True)
구매_한도_20만원_연령별_2

/usr/local/lib/python3.8/dist-packages/pandas/core/generic.py:4150: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


구매건수  총금액(A+B)
나이대 이용자번호          나이   성별                
90대 20082431816859 90.0 F     11   2200000
    21050351136205 92.0 F      6   1200000
    21070110069510 97.0 F      5   1000000
    21070210858435 90.0 F      3    600000
    19102203088420 93.0 F      2    400000
80대 20093045994289 80.0 M     13   2400000
    20093046004820 80.0 F     14   2400000
    21010976659470 82.0 M     12   2400000
    21063009428583 84.0 F     12   2400000
    20011607614476 83.0 F     11   2200000
70대 20113063624542 78.0 M     13   2500000
    19090401110826 72.0 F     12   2400000
    19090901508553 74.0 M     12   2400000
    19121005099031 76.0 M     12   2400000
    19121205170038 71.0 F     12   2400000
60대 20092744387731 67.0 F     15   2600000
    19082800616309 60.0 F     13   2500000
    20092643975547 66.0 F     13   2401000
    19082900670858 63.0 F     12   2400000
    19083100805068 63.0 M     12   2400000
50대 19082600289258 58.0 F     14   2600000
    20092241049419 54.0 F     13   2600000
    21010475042007 50.0 M     13   2600000
    21041640298675 57.0 F     13   2600000
    20090334821599 51.0 F     16   2580000
40대 20092140309332 46.0 F     24   2730000
    20013110463479 48.0 F     15   2620000
    20032721205325 45.0 F     15   2600000
    20042771039589 44.0 F     13   2600000
    20060408287947 46.0 F     13   2600000
30대 20062513751480 39.0 F     13   2600000
    20102653326849 38.0 F     13   2600000
    21022005067607 32.0 F     13   2600000
    20031816666726 37.0 F     15   2501000
    20021812851258 36.0 F     13   2500000
20대 19082100150057 28.0 F     12   2400000
    19082800568095 28.0 F     12   2400000
    19120204499144 29.0 F     12   2400000
    19122105680773 29.0 M     12   2400000
    20020410944793 22.0 M     12   2400000
10대 20052805543559 19.0 M     12   2400000
    20072524302478 18.0 M     12   2400000
    20082131133369 19.0 M     12   2400000
    20092945722688 19.0 M     13   2400000
    20101550321524 19.0 M     12   2400000

# 월별 이용자별 구매여부 확인

In [ ]:
구매_한도_20만원

,Unnamed: 0,거래번호,상품권,구매처,구매일,구매시,이용자번호,이용자명,전화번호,총금액(A+B),...,지원금(B),성별,생년월일,구매년도,구매월,구매일(d),구매시(h),나이,나이대,구매년월
415515,415515,PKP0103610278,김해사랑상품권,비즈플레이(경남),2021-08-15,18:37,20120163760034,김은하,1.049464e+09,200000,...,20000,F,197602.0,2021,8,15,18,47.0,40대,2021-8
415516,415516,PKP0103567297,김해사랑상품권,농협은행,2021-08-15,14:08,19082900660360,김명희,1.030892e+09,30000,...,3000,F,196511.0,2021,8,15,14,58.0,50대,2021-8
415517,415517,PKP0103196858,김해사랑상품권,비즈플레이,2021-08-13,18:15,21062406984361,류성훈,1.038472e+09,25000,...,2500,M,197211.0,2021,8,13,18,51.0,50대,2021-8
415518,415518,PKP0103196570,김해사랑상품권,비즈플레이,2021-08-13,18:14,21062406984361,류성훈,1.038472e+09,15000,...,1500,M,197211.0,2021,8,13,18,51.0,50대,2021-8
415519,415519,PKP0103160382,김해사랑상품권,비즈플레이,2021-08-13,15:10,21010776106714,이혜정,1.085258e+09,200000,...,20000,F,195703.0,2021,8,13,15,66.0,60대,2021-8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1040694,1040694,PKP0204040632,김해사랑상품권,비즈플레이,2022-07-01,11:01,20090936355925,구미정,1.092592e+09,200000,...,20000,F,197809.0,2022,7,1,11,45.0,40대,2022-7
1040695,1040695,PKP0204040625,김해사랑상품권,비즈플레이,2022-07-01,11:01,20041957483163,김선희,1.085011e+09,200000,...,20000,F,198206.0,2022,7,1,11,41.0,40대,2022-7
1040696,1040696,PKP0204040607,김해사랑상품권,농협은행,2022-07-01,11:01,20042365732963,박인숙,1.024182e+09,200000,...,20000,F,197902.0,2022,7,1,11,44.0,40대,2022-7
1040697,1040697,PKP0204040605,김해사랑상품권,농협은행,2022-07-01,11:01,19082700301763,임은주,1.093295e+09,200000,...,20000,F,197711.0,2022,7,1,11,46.0,40대,2022-7


In [13]:
구매여부 = 구매_한도_20만원.groupby(['이용자번호', '구매년월']).count()
구매여부.reset_index(inplace=True)
구매_여부 = 구매여부[['이용자번호', '구매년월', '상품권']]
구매_여부.rename(columns={'상품권':'구매여부'}, inplace=True)
구매_여부

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,이용자번호,구매년월,구매여부
0,19081700008385,2021-7,1
1,19081700008385,2021-8,1
2,19081700011038,2021-12,1
3,19081800011980,2021-8,1
4,19081800011980,2022-1,1
...,...,...,...
518785,22080175038360,2022-8,1
518786,22080175039768,2022-8,1
518787,22080275274269,2022-8,1
518788,22080475541765,2022-8,1


In [14]:
구매_여부_ = 구매_여부.pivot(index='이용자번호', columns='구매년월', values='구매여부')
구매_여부_ = 구매_여부_.fillna(0)

In [21]:
구매_여부_

구매년월,2021-10,2021-12,2021-7,2021-8,2022-1,2022-2,2022-3,2022-4,2022-5,2022-6,2022-7,2022-8
이용자번호,,,,,,,,,,,,
19081700008385,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19081700011038,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19081800011980,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0
19081800012182,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19081900014444,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
22080175038360,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
22080175039768,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
22080275274269,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


# 김해 공공배달앱 상품권 구매내역

## 데이터 전처리

In [ ]:
배달앱 = pd.read_excel('/content/구매내역_김해 공공배달앱 상품권.xlsx')
배달앱

In [23]:
배달앱.drop('Unnamed: 0', axis=1, inplace=True)
배달앱

,상품권,구매처,구매일,구매월,구매시,이용자번호,총금액(A+B),고객구매금(A),지원금(B),성별,생년월일,연령대
0,김해 공공배달앱 상품권,비즈플레이(경남),2022-06-24,6,14:10,20100146147015,50000,45000,5000,M,196608.0,50대
1,김해 공공배달앱 상품권,비즈플레이,2022-06-05,6,10:12,21041137945124,160000,144000,16000,M,197106.0,50대
2,김해 공공배달앱 상품권,비즈플레이(경남),2022-06-04,6,08:10,20100146147015,50000,45000,5000,M,196608.0,50대
3,김해 공공배달앱 상품권,비즈플레이,2022-06-03,6,21:27,21041137945124,40000,36000,4000,M,197106.0,50대
4,김해 공공배달앱 상품권,비즈플레이,2022-06-03,6,15:10,20062413648105,10000,9000,1000,F,198205.0,40대
...,...,...,...,...,...,...,...,...,...,...,...,...
10507,김해 공공배달앱 상품권,비즈플레이,2022-07-01,7,09:02,21122708635719,200000,180000,20000,F,200712.0,10대
10508,김해 공공배달앱 상품권,비즈플레이,2022-07-01,7,09:02,20041856071887,200000,180000,20000,F,198507.0,30대
10509,김해 공공배달앱 상품권,비즈플레이,2022-07-01,7,09:02,20101550242427,200000,180000,20000,M,199012.0,30대
10510,김해 공공배달앱 상품권,비즈플레이,2022-07-01,7,09:02,20011407251293,200000,180000,20000,F,198402.0,30대


In [24]:
배달앱['구매년도']=배달앱['구매일'].str.split('-', expand=True)[0]
배달앱['구매월']=배달앱['구매일'].str.split('-', expand=True)[1]
배달앱['구매일(d)']=배달앱['구매일'].str.split('-', expand=True)[2]
배달앱['구매시(h)']=배달앱['구매시'].str.split(':', expand=True)[0]

In [25]:
배달앱 = 배달앱.astype({'이용자번호':'str'})

In [26]:
배달앱 = 배달앱.astype({'생년월일':'str'})
배달앱['나이1']=배달앱['생년월일'].str.split('.', expand=True)[0]
배달앱['나이2'] = 배달앱['나이1'].str[0:4]
배달앱 = 배달앱.astype({'나이2':'float'})
배달앱['연령'] = (2022-배달앱['나이2'])+1
배달앱 = 배달앱.astype({'연령':'str'})
배달앱['연령대'] = 배달앱['연령'].str[0]+'0대'
배달앱 = 배달앱.drop(['나이1', '나이2'], axis = 1)
배달앱

,상품권,구매처,구매일,구매월,구매시,이용자번호,총금액(A+B),고객구매금(A),지원금(B),성별,생년월일,연령대,구매년도,구매일(d),구매시(h),연령
0,김해 공공배달앱 상품권,비즈플레이(경남),2022-06-24,06,14:10,20100146147015,50000,45000,5000,M,196608.0,50대,2022,24,14,57.0
1,김해 공공배달앱 상품권,비즈플레이,2022-06-05,06,10:12,21041137945124,160000,144000,16000,M,197106.0,50대,2022,05,10,52.0
2,김해 공공배달앱 상품권,비즈플레이(경남),2022-06-04,06,08:10,20100146147015,50000,45000,5000,M,196608.0,50대,2022,04,08,57.0
3,김해 공공배달앱 상품권,비즈플레이,2022-06-03,06,21:27,21041137945124,40000,36000,4000,M,197106.0,50대,2022,03,21,52.0
4,김해 공공배달앱 상품권,비즈플레이,2022-06-03,06,15:10,20062413648105,10000,9000,1000,F,198205.0,40대,2022,03,15,41.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10507,김해 공공배달앱 상품권,비즈플레이,2022-07-01,07,09:02,21122708635719,200000,180000,20000,F,200712.0,10대,2022,01,09,16.0
10508,김해 공공배달앱 상품권,비즈플레이,2022-07-01,07,09:02,20041856071887,200000,180000,20000,F,198507.0,30대,2022,01,09,38.0
10509,김해 공공배달앱 상품권,비즈플레이,2022-07-01,07,09:02,20101550242427,200000,180000,20000,M,199012.0,30대,2022,01,09,33.0
10510,김해 공공배달앱 상품권,비즈플레이,2022-07-01,07,09:02,20011407251293,200000,180000,20000,F,198402.0,30대,2022,01,09,39.0


## 김해 공공배달앱 상품권 관련 통계들
- 김해 공공배달앱 상품권은 2022년 05월부터 판매

In [31]:
# 월별 구매내역
배달앱_월별_구매내역 = 배달앱.groupby(['구매년도', '구매월']).agg({'상품권':'count', '총금액(A+B)':'sum', '고객구매금(A)':'sum', '지원금(B)':'sum'})
배달앱_월별_구매내역.rename(columns={'상품권':'구매건수'}, inplace=True)
배달앱_월별_구매내역

구매건수   총금액(A+B)   고객구매금(A)    지원금(B)
구매년도 구매월                                      
2022 05   5409  669590000  602631000  66959000
     06    853  116420000  104778000  11642000
     07   4172  467015000  420313500  46701500
     08     78   10000000    9000000   1000000

In [33]:
# 연령별 / 성별 구매내역
배달앱_연령별_성별_구매내역 = 배달앱.groupby(['연령대', '성별']).agg({'상품권':'count', '총금액(A+B)':'sum', '고객구매금(A)':'sum', '지원금(B)':'sum'})
배달앱_연령별_성별_구매내역.rename(columns={'상품권':'구매건수'}, inplace=True)
배달앱_연령별_성별_구매내역

구매건수   총금액(A+B)   고객구매금(A)    지원금(B)
연령대 성별                                      
10대 F     20    3705000    3334500    370500
    M     22    2985000    2686500    298500
20대 F    362   28470000   25623000   2847000
    M    148   15795000   14215500   1579500
30대 F   1972  195655000  176089500  19565500
    M    915  101055000   90949500  10105500
40대 F   3555  433540000  390186000  43354000
    M   1703  229450000  206505000  22945000
50대 F    876  115695000  104125500  11569500
    M    615   86595000   77935500   8659500
60대 F    141   23815000   21433500   2381500
    M    102   15595000   14035500   1559500
70대 F     41    4675000    4207500    467500
    M     30    4715000    4243500    471500
80대 F      2     300000     270000     30000
    M      3     580000     522000     58000
90대 F      1     100000      90000     10000

In [19]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
cd /content/drive/MyDrive/Github/code

/content/drive/MyDrive/Github/code


In [21]:
!git config --global user.email 'qls5528@gmail.com'
!git config --global user.name 'janghyebinee'

!git add buy_code.ipynb

!git commit -m '구매내역 코드'
!git push

fatal: pathspec 'buy_code.ipynb' did not match any files
On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
	modified:   test.ipynb

Untracked files:
	buy_code

no changes added to commit
Everything up-to-date
